# Loading results from multinest

In [ ]:
# Setup packages
%run load_results_multinest.py

In [ ]:
print(df.head(5))

In [ ]:
df.head(5).to_csv('test.csv', index=False)

In [ ]:
df_read = pd.read_csv('test.csv')


In [ ]:
np.shape(df_read) == np.shape(df.head(5))

In [ ]:
df.head(5)[0:1]

In [ ]:
df['tol'].values

In [ ]:
# for tol in np.unique(df['tol'].values):
#     if not np.isnan(tol):
#         print(tol, 
#               np.mean(df['config_fit_time'][df['tol']==tol]).astype(np.int), 
#               np.std(df['config_fit_time'][df['tol']==tol]).astype(np.int))

In [ ]:
finished = df[['mw', 'config_sigma', 'config_detector', 'config_notes', 'n_fit_parameters', 'config_nlive', 'config_save_intermediate', 'config_earth_shielding']].copy()
def strip_n(n):
    if n.endswith('038'):  n = n[:-1]
    elif n.endswith('_n12') or n.endswith('_n16'): n = n[:-4]
    return n
notes = [strip_n(n) for n in finished.config_notes.values]
finished['config_notes'] = notes
finished.to_csv('../finished_sims.csv', index=False)
finished

In [ ]:
# def df_to_sum(df_):
#     res = []
#     for name in ['mw', 'config_sigma']:
#         res['mw'] 
    
#     ['config_detector', 'config_notes', 'n_fit_parameters',
#                  'config_nlive', 'config_save_intermediate', 'config_earth_shielding']:
        

In [ ]:
delete_empty(no_result)

In [ ]:
mask = df['config_nlive'] < 500
df[mask]

In [ ]:
delete_with_mask(df, mask)

In [ ]:
all_notes = np.unique(df.config_notes.values)
for note in all_notes:
    print(f'{np.sum(df["config_notes"] == note)}\t{note}')

In [ ]:
mask = df['config_notes'] == 'migdal_04_038'
df_copy = df[~mask].copy()
df_copy

In [ ]:
def three_confidence_plot(items, text_box=False,
                        bin_range=None, nsigma=1,
                        smoothing=None,
                        nbins=50):
    if bin_range == None:
        bin_range = [results[items[0]]['config']['prior']['log_mass']['range'],
                     results[items[0]]['config']['prior']['log_cross_section']['range']
                     ]
    hists = {}
    for k, item in enumerate(items):  # , 78, 110
        x, y = get_p_i(item)
        # Make a 2d normed histogram
        hists[k], xedges, yedges = np.histogram2d(x, y, bins=nbins, range=bin_range, normed=True)
        X, Y = bin_center(xedges, yedges)
        if smoothing:
            hists[k] = sp.ndimage.filters.gaussian_filter(
                hists[k].T,
                [np.sqrt(nbins) / 10, np.sqrt(nbins) / 10],
                mode='constant')
        else:
            hists[k] = hists[k].T
        _confidence_plot(item, X, Y, hists[k], bin_range, text_box=text_box, nsigma=nsigma, cmap = [cm.viridis, cm.gnuplot2_r][k])
    if len(items) == 2:
        H = hists[0] * hists[1]
    else:
        raise ValueError(f'Len items is {len(items)}')
    H = H / np.sum(H)
    
    _confidence_plot(item, X, Y, H, bin_range, text_box=text_box, nsigma=nsigma+1)

    

In [ ]:
tmp = list(dict(a=1).keys()) + ['n']
tmp

In [ ]:
len(df_copy[df_copy['config_detector']=='Ge_migd_iZIP_bg'].item.values)

In [ ]:
for i in range(40):
    it = df_copy[df_copy['config_detector']=='Ge_migd_iZIP_bg'].item.values[i]
    print([it, match_other_item(it)])
    if match_other_item(it) == None:
        continue
    three_confidence_plot([it, match_other_item(it)])
    plt.show()

In [ ]:
mask = [val in [148, 201] for val in df_copy['item'].values ]
df_copy[mask]

In [ ]:
three_confidence_plot([148, 201])

In [ ]:
one_confidence_plot(148)

In [ ]:
one_confidence_plot(201)

In [ ]:
results[18]['config']

In [ ]:
results[19]['config']

In [ ]:
dfs = []
for mass in np.unique(df_copy['mw']):
    mask1 = df_copy['mw'] == mass
    for sigma in np.unique(df_copy[mask1]['config_sigma']):
        mask2 = df_copy[mask1]['config_sigma'] == sigma
        sel3 = df_copy[mask1][mask2]['config_detector']
        for det in np.unique(sel3):
            mask3 = sel3 == det
            sel4 = df_copy[mask1][mask2][mask3]['n_fit_parameters']
            print(mass, sigma, det, '\n\n\n')
            for n_ in np.unique(sel4):
                mask4 = sel4 == n_
                sel5 = df_copy[mask1][mask2][mask3][mask3][mask4]['config_earth_shielding']
                for shield in np.unique(sel5):                    
                    mask5 = sel5 == shield
                    sel6 = df_copy[mask1][mask2][mask3][mask3][mask4][mask5]['config_save_intermediate']
                    for intm in np.unique(sel6):
                        mask6 = sel6 == intm
                        this_df = df_copy[mask1][mask2][mask3][mask3][mask4][mask5][mask6]
#                         print(mass, sigma, det, n_, shield, intm, len(this_df))
                        if len(this_df) > 1:
                            del_mask = this_df_copy['config_start'] != np.max(this_df_copy['config_start'])
#                             delete_with_mask(this_df, del_mask, True)
                            dfs.append(this_df)
#                         for k in this_df.sort_values('config_start')['item'].values:
#                             overlay_hist_confidence_info(k)

In [ ]:
dfs = []
for mass in np.unique(df_copy['mw']):
    mask1 = df_copy['mw'] == mass
    for sigma in np.unique(df_copy[mask1]['config_sigma']):
        mask2 = df_copy[mask1]['config_sigma'] == sigma
        sel3 = df_copy[mask1][mask2]['config_detector']
        for det in np.unique(sel3):
            mask3 = sel3 == det
            sel4 = df_copy[mask1][mask2][mask3]['n_fit_parameters']
            print(mass, sigma, det, '\n\n\n')
            for n_ in np.unique(sel4):
                mask4 = sel4 == n_
                sel5 = df_copy[mask1][mask2][mask3][mask3][mask4]['config_earth_shielding']
                for shield in np.unique(sel5):                    
                    mask5 = sel5 == shield
                    sel6 = df_copy[mask1][mask2][mask3][mask3][mask4][mask5]['config_save_intermediate']
                    for intm in np.unique(sel6):
                        mask6 = sel6 == intm
                        this_df = df_copy[mask1][mask2][mask3][mask3][mask4][mask5][mask6]
                        if len(this_df) > 1:
                            del_mask = this_df_copy['config_start'] != np.max(this_df_copy['config_start'])
                            # delete_with_mask(this_df, del_mask, True)
                            dfs.append(this_df)
                        df_iZIP = this_df[this_df['config_detector'] == 'Ge_migd_iZIP_bg']
                        for item in df_iZIP['item'].values:
                            print('Ge')
                            other_item = match_other_item(item)
                            print(item, other_item)
                            if other_item:
                                sel_items = [item, match_other_item(item)]
                                two_confidence_plot(sel_items, nbins = 200, smoothing = True)
                                plt.show()
                            else:
                                sel_items = [item]
                                one_confidence_plot(item)
                                plt.show()

                            print('Xe')
                            diff_det = match_other_item(item, diff_det_type = True)
                            if np.isscalar(diff_det):
                                print(diff_det)
                                bin_range = get_binrange(diff_det)
                                print(bin_range)
                                one_confidence_plot(diff_det)
                                plt.xlim(*bin_range[0]), plt.ylim(*bin_range[1])
                                plt.show()
                                

In [ ]:
# df_iZIP = df_copy[df_copy['config_detector'] == 'Ge_migd_iZIP_bg']

# for item in df_iZIP['item'].values:
#     print('Ge')
#     other_item = match_other_item(item)
#     print(item, other_item)
#     if other_item:
#         sel_items = [item, match_other_item(item)]
#     else:
#         sel_items = [item]
#     print(sel_items)
#     bin_range = get_binrange(item)
#     two_confidence_plot(sel_items, bin_range=bin_range, nbins = 200, smoothing = True)
#     plt.xlim(*bin_range[0]), plt.ylim(*bin_range[1])
#     plt.show()
    
#     print('Xe')
#     diff_det = match_other_item(item, diff_det_type = True)
#     if diff_det:
#         print(diff_det)
#         bin_range = get_binrange(diff_det)
#         print(bin_range)
#         two_confidence_plot([diff_det], bin_range=bin_range, nbins = 200, smoothing = True)
#         plt.xlim(*bin_range[0]), plt.ylim(*bin_range[1])
#         plt.show()